<a href="https://colab.research.google.com/github/Develtomas/Python_course/blob/main/mt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv("hdfs:///user/spark/covid/cov.csv")
df.printSchema()

### Task1


In [ ]:
from pyspark.sql.functions import *

df.select("iso_code", "location", (col("total_cases_per_million")/10000).alias("%")).where(col("date")=="2021-03-31").sort(col("%").desc()).show(15)

```
+--------+-------------+------------------+
|iso_code|     location|                 %|
+--------+-------------+------------------+
|     AND|      Andorra|        15.5439073|
|     MNE|   Montenegro|14.523725399999998|
|     CZE|      Czechia|        14.3088484|
|     SMR|   San Marino|        13.9371796|
|     SVN|     Slovenia|10.370805800000001|
|     LUX|   Luxembourg|         9.8473424|
|     ISR|       Israel|          9.625106|
|     USA|United States|          9.203011|
|     SRB|       Serbia|         8.8263286|
|     BHR|      Bahrain|         8.4888601|
|     PAN|       Panama|         8.2287391|
|     PRT|     Portugal|         8.0586997|
|     EST|      Estonia|         8.0226816|
|     SWE|       Sweden|         7.9697443|
|     LTU|    Lithuania|         7.9388647|
+--------+-------------+------------------+
only showing top 15 rows
```

###Task2


In [ ]:
from pyspark.sql.window import Window

df.where((to_date(col("date")) >= "2021-03-29") & (to_date(col("date")) <= "2021-03-31"))\
    .withColumn("max_cases", max(col("new_cases")).over(Window.partitionBy("location")))\
    .select("date", "location", "new_cases")\
    .where(col("max_cases") == col("new_cases"))\
    .where(~col("location").isin(["World", "Europe", "European Union", "Asia", "South America", "North America"]))\
    .sort(col("max_cases").desc()).show(10)

```
+-------------------+-------------+---------+
|               date|     location|new_cases|
+-------------------+-------------+---------+
|2021-03-31 00:00:00|       Brazil|  90638.0|
|2021-03-31 00:00:00|        India|  72330.0|
|2021-03-29 00:00:00|United States|  69429.0|
|2021-03-31 00:00:00|       France|  59054.0|
|2021-03-31 00:00:00|       Turkey|  39302.0|
|2021-03-31 00:00:00|       Poland|  32891.0|
|2021-03-31 00:00:00|      Germany|  25014.0|
|2021-03-31 00:00:00|        Italy|  23887.0|
|2021-03-30 00:00:00|       Sweden|  16427.0|
|2021-03-31 00:00:00|    Argentina|  16056.0|
+-------------------+-------------+---------+
only showing top 10 rows
```



### Task3



In [ ]:
from pyspark.sql.window import Window

df.where((to_date(col("date")) >= "2021-03-28") & (to_date(col("date")) <= "2021-03-31") & (col("location") == "Russia"))\
    .withColumn("yesterday_cases", lag("new_cases").over(Window.orderBy("date")))\
    .select("date", "yesterday_cases", "new_cases", (col("new_cases")-col("yesterday_cases")).alias("delta"))\
    .where((to_date(col("date")) > "2021-03-28"))\
    .show()

```
+-------------------+---------------+---------+------+
|               date|yesterday_cases|new_cases| delta|
+-------------------+---------------+---------+------+
|2021-03-29 00:00:00|         8979.0|   8589.0|-390.0|
|2021-03-30 00:00:00|         8589.0|   8162.0|-427.0|
|2021-03-31 00:00:00|         8162.0|   8156.0|  -6.0|
+-------------------+---------------+---------+------+
```